In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt


In [3]:

import chardet
path = '../LR2/spotify-2023.csv'
with open(path, 'rb') as f:
    result = chardet.detect(f.read())
    encoding = result['encoding']

data = pd.read_csv(path, encoding=encoding)
data = data.drop(574)
data = data.dropna()
print(data.count());

track_name              816
artist(s)_name          816
artist_count            816
released_year           816
released_month          816
released_day            816
in_spotify_playlists    816
in_spotify_charts       816
streams                 816
in_apple_playlists      816
in_apple_charts         816
in_deezer_playlists     816
in_deezer_charts        816
in_shazam_charts        816
bpm                     816
key                     816
mode                    816
danceability_%          816
valence_%               816
energy_%                816
acousticness_%          816
instrumentalness_%      816
liveness_%              816
speechiness_%           816
dtype: int64


# Было решено предсказывать danceability

## Задание 2

### ввиду того, что предсказываем процент danceability, то очевдно, что это задача регрессии

In [4]:
selected_features = ['artist_count', 'released_year', 'streams', 'in_apple_playlists']
target_variable = 'danceability_%'
X = data[selected_features]
X['streams']=X['streams'].astype(float)
X['artist_count']=X['artist_count'].astype(float)
X['released_year']=X['released_year'].astype(float)
X['in_apple_playlists']=X['in_apple_playlists'].astype(float)
Y = data[target_variable]

/var/folders/tf/4vf66cxs0f747m9rb3jpvyhc0000gn/T/ipykernel_93546/1403072990.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['streams']=X['streams'].astype(float)
/var/folders/tf/4vf66cxs0f747m9rb3jpvyhc0000gn/T/ipykernel_93546/1403072990.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['artist_count']=X['artist_count'].astype(float)
/var/folders/tf/4vf66cxs0f747m9rb3jpvyhc0000gn/T/ipykernel_93546/1403072990.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

In [5]:

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=17)

In [6]:
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train[selected_features])

X_test_scaled = scaler.transform(X_test[selected_features])

X.loc[X_train.index, selected_features] = X_train_scaled
X.loc[X_test.index, selected_features] = X_test_scaled




In [7]:
X.std()

artist_count          1.028629
released_year         1.013628
streams               1.012255
in_apple_playlists    1.037737
dtype: float64

In [8]:
X['streams'].mean()

-0.02232481104313965

# Задание 4, 5

In [9]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor

from sklearn.linear_model import SGDRegressor

from sklearn.metrics import mean_absolute_error,  mean_squared_error



In [10]:
def getMetrics(test,predict):
    MAE  = mean_absolute_error(test, predict)
    print(f"MAE: {MAE:.2f}")

    RMSE = np.sqrt(mean_squared_error(test, predict))
    print(f"RMSE: {RMSE:.2f}")

    MAPE = np.mean(np.abs((test - predict) / test)) * 100
    print(f"MAPE: {MAPE:.2f}%")

## тест сет

обучить на трейне

In [11]:
linClassifier  = LinearRegression();

print(f"{'Тест сет':–^25}")


linClassifier.fit(X_train_scaled, y_train)
lr_pred = linClassifier.predict(X_test_scaled)
getMetrics(y_test, lr_pred)

print(f"{'Трейн сет':–^25}")
linClassifier  = LinearRegression();
linClassifier.fit(X_train_scaled, y_train)

logr_pred_train = linClassifier.predict(X_train_scaled)

getMetrics(y_train, logr_pred_train)


––––––––Тест сет–––––––––
MAE: 12.75
RMSE: 15.76
MAPE: 22.00%
––––––––Трейн сет––––––––
MAE: 10.96
RMSE: 13.54
MAPE: 18.68%


In [12]:
knn_regr = KNeighborsRegressor(5)
knn_regr.fit(X_train_scaled, y_train)
knn_regr_pred = knn_regr.predict(X_test_scaled)

print(f"{'Тест сет':–^25}")

getMetrics(y_test, knn_regr_pred)

print(f"{'Трейн сет':–^25}")

knn_regr = KNeighborsRegressor(5)
knn_regr.fit(X_train_scaled, y_train)
knn_regr_pred = knn_regr.predict(X_test_scaled)

getMetrics(y_test, knn_regr_pred)

––––––––Тест сет–––––––––
MAE: 12.81
RMSE: 16.30
MAPE: 22.08%
––––––––Трейн сет––––––––
MAE: 12.81
RMSE: 16.30
MAPE: 22.08%


# KNN, k=10

In [13]:
knn_regr = KNeighborsRegressor(10)
knn_regr.fit(X_train_scaled, y_train)
knn_regr_pred = knn_regr.predict(X_test_scaled)

print(f"{'Тест сет':–^25}")

getMetrics(y_test, knn_regr_pred)

print(f"{'Трейн сет':–^25}")

knn_regr = KNeighborsRegressor(10)
knn_regr.fit(X_train_scaled, y_train)
knn_regr_pred = knn_regr.predict(X_test_scaled)

getMetrics(y_test, knn_regr_pred)

––––––––Тест сет–––––––––
MAE: 12.14
RMSE: 15.53
MAPE: 21.36%
––––––––Трейн сет––––––––
MAE: 12.14
RMSE: 15.53
MAPE: 21.36%


In [14]:
treeRegr  = DecisionTreeRegressor(max_depth=10)


treeRegr.fit(X_train_scaled, y_train)
treeRegr_pred = treeRegr.predict(X_test_scaled)
print(f"{'Тест сет':–^25}")

getMetrics(y_test, treeRegr_pred)

print(f"{'Трейн сет':–^25}")

treeRegr  = DecisionTreeRegressor(max_depth=10)


treeRegr.fit(X_train_scaled, y_train)
treeRegr_pred = treeRegr.predict(X_train_scaled)

getMetrics(y_train, treeRegr_pred)

––––––––Тест сет–––––––––
MAE: 13.97
RMSE: 18.29
MAPE: 23.21%
––––––––Трейн сет––––––––
MAE: 5.30
RMSE: 8.28
MAPE: 8.49%


In [15]:
forestRegr  = RandomForestRegressor()


forestRegr.fit(X_train_scaled, y_train)
forestRegr_pred = forestRegr.predict(X_test_scaled);
print(f"{'Тест сет':–^25}")

getMetrics(y_test, forestRegr_pred)


print(f"{'Трейн сет':–^25}")

forestRegr  = RandomForestRegressor()


forestRegr.fit(X_train_scaled, y_train)
forestRegr_pred = forestRegr.predict(X_train_scaled);

getMetrics(y_train, forestRegr_pred)

––––––––Тест сет–––––––––
MAE: 12.58
RMSE: 15.82
MAPE: 21.33%
––––––––Трейн сет––––––––
MAE: 4.17
RMSE: 5.33
MAPE: 6.99%


## Попытки улучшить модель

In [25]:
from sklearn.preprocessing import PolynomialFeatures

for power in range(1,10):
    poly = PolynomialFeatures(degree=power)

    X_train_poly = poly.fit_transform(X_train[selected_features])
    
    regr = LinearRegression()

    regr.fit(X_train_poly, y_train)
    X_test_poly = poly.transform(X_test[selected_features])
    pred_test = regr.predict(X_test_poly)
    print(f"Степень {power} для тестового множества:")
    getMetrics(y_test, pred_test)
    print()
    pred_train = regr.predict(X_train_poly)
    print(f"Степень {power} для тренировочного множества:")
    getMetrics(y_train, pred_train)
    
    print(f"{f'–':–^25}")
    


Степень 1 для тестового множества:
MAE: 12.75
RMSE: 15.76
MAPE: 22.00%

Степень 1 для тренировочного множества:
MAE: 10.96
RMSE: 13.54
MAPE: 18.68%
–––––––––––––––––––––––––
Степень 2 для тестового множества:
MAE: 12.73
RMSE: 15.79
MAPE: 21.98%

Степень 2 для тренировочного множества:
MAE: 10.92
RMSE: 13.53
MAPE: 18.62%
–––––––––––––––––––––––––
Степень 3 для тестового множества:
MAE: 13.04
RMSE: 15.95
MAPE: 22.64%

Степень 3 для тренировочного множества:
MAE: 11.32
RMSE: 14.01
MAPE: 19.45%
–––––––––––––––––––––––––
Степень 4 для тестового множества:
MAE: 13.13
RMSE: 16.05
MAPE: 22.80%

Степень 4 для тренировочного множества:
MAE: 11.40
RMSE: 14.10
MAPE: 19.62%
–––––––––––––––––––––––––
Степень 5 для тестового множества:
MAE: 13.69
RMSE: 18.19
MAPE: 23.49%

Степень 5 для тренировочного множества:
MAE: 11.45
RMSE: 14.15
MAPE: 19.71%
–––––––––––––––––––––––––
Степень 6 для тестового множества:
MAE: 14.68
RMSE: 27.51
MAPE: 24.65%

Степень 6 для тренировочного множества:
MAE: 11.49
RMSE: 1

## Оценка

Самая лучшая модель оказалась KNNRegr с k=10 с абсолютной ошибкой MAPE =
 21.36%